In [3]:
import torch
import numpy as np
from sympy import shape

# 创建一个零维张量
tensor0d = torch.tensor(1)
# 创建一个一维张量
tensor1d = torch.tensor([1,2,3])
# 创建一个二维张量
tensor2d = torch.tensor([[1,2],
                         [3,4]])
# 创建一个三维张量
tensor3d = torch.tensor([[[1,2],[3,4]],
                         [[5,6],[7,8]]])
ary3d = np.array([[[1,2],[3,4]],
                         [[5,6],[7,8]]])

tensor3d_2 = torch.tensor(ary3d)  # 复制 NumPy 数组
tensor3d_3 = torch.from_numpy(ary3d)   # 与 NumPy 数组共享内存

print(tensor3d_2, tensor3d_3)

tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]]) tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])


In [4]:
ary3d[0,0,0] = 999
print(tensor2d)

tensor([[1, 2],
        [3, 4]])


In [5]:
print(tensor3d_3)

tensor([[[999,   2],
         [  3,   4]],

        [[  5,   6],
         [  7,   8]]])


In [6]:
tensorId = torch.tensor([1,2,3])
print(tensorId.dtype)

torch.int64


In [7]:
floatvec = torch.tensor([1.0,2.0,3.0])
print(floatvec.dtype)

torch.float32


In [8]:
floatvec = tensorId.to(torch.float)
print(floatvec.dtype)

torch.float32


In [11]:
tensor2d = torch.tensor([[1,2,3],[3,4,5]])
tensor2d

tensor([[1, 2, 3],
        [3, 4, 5]])

In [12]:
tensor2d.shape

torch.Size([2, 3])

In [13]:
tensor2d.reshape([3,2])

tensor([[1, 2],
        [3, 3],
        [4, 5]])

In [14]:
tensor2d.view(3,2)

tensor([[1, 2],
        [3, 3],
        [4, 5]])

In [15]:
tensor2d.T

tensor([[1, 3],
        [2, 4],
        [3, 5]])

In [16]:
tensor2d.matmul(tensor2d.T)

tensor([[14, 26],
        [26, 50]])

In [17]:
tensor2d @ tensor2d.T

tensor([[14, 26],
        [26, 50]])

In [19]:
# 将模型视为计算图
import torch.nn.functional as F

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2])
b = torch.tensor([0.0])

z = x1 * w1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a,y)
print(loss)

tensor(0.0852)


In [20]:
# A.4 轻松实现自动微分
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad = True)
b = torch.tensor([0.0], requires_grad = True)

z = x1 * w1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)

gard_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

print(gard_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [21]:
loss.backward()

print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


In [23]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            torch.nn.Linear(20, num_outputs),
        )
    def forward(self, x):
        logits = self.layers(x)
        return logits

In [24]:
model = NeuralNetwork(50, 3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [25]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("total number of parameters: {}".format(num_params))

total number of parameters: 2213


In [26]:
print(model.layers[0].weight)

Parameter containing:
tensor([[ 0.1015, -0.0857, -0.1257,  ..., -0.0539,  0.0374,  0.0141],
        [ 0.1219,  0.0175,  0.0456,  ...,  0.1173,  0.1406, -0.1370],
        [-0.0168,  0.0841,  0.0817,  ...,  0.0731,  0.0398,  0.0958],
        ...,
        [-0.1079,  0.0787,  0.0681,  ...,  0.0674,  0.0705, -0.1261],
        [ 0.0134,  0.0019, -0.0300,  ..., -0.0851, -0.1235, -0.0402],
        [-0.0540, -0.0823, -0.0700,  ..., -0.0796,  0.1388, -0.0054]],
       requires_grad=True)


In [27]:
torch.manual_seed(123)

model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [33]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [39]:
torch.manual_seed(123)

X = torch.rand((1,50))
out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [ ]:
with torch.no_grad():
    out = model(X)
print(out)